In [1]:
# !pip install pandas
# !pip install seaborn
# !pip install sklearn
# !pip install xgboost

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn.preprocessing

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
test['Transported'] = 'X'

In [4]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [5]:

df = train.append(test,ignore_index=True)
# df = df[df['Cabin'].notna()]
df['Cabin'].fillna('F/603/S', inplace = True)
# k = dict(df['Name'].value_counts())
# k[np.nan]=1
# df['Name_f'] = [k[df['Name'][i]] for i in range(len(df))]
df = df.reset_index(drop=True)
df['Cabin_0'] = [str(df['Cabin'][i]).split('/')[0] for i in range(len(df))]
df['Cabin_1'] = [str(df['Cabin'][i]).split('/')[1] for i in range(len(df))]
df['Cabin_2'] = [str(df['Cabin'][i]).split('/')[2] for i in range(len(df))]
df['Passenger'] = [int(str(df['PassengerId'][i]).split('_')[1]) for i in range(len(df))]
df['Passenger1'] = [int(str(df['PassengerId'][i]).split('_')[0]) for i in range(len(df))]
df['VIP'].fillna(False, inplace = True)
df['Age'].fillna(df['Age'].mean(), inplace = True)
df['VIP'] = df['VIP'].astype(int)
df['Cabin_1'] = df['Cabin_1'].astype(int)
factor = 1
for i in range(int(max(df['Age'])/factor)):
    column_name = "Age_" + str(i*factor) + "_" + str(factor*i+factor)
    arr = []
    for j in range(len(df)):
        if(int(df['Age'][j])>(i*factor) and int(df['Age'][j])<=(factor*i+factor)):
            arr.append(1)
        else:
            arr.append(0)
    df[column_name] = arr
# 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Cabin_1'
df['RoomService'].fillna(df['RoomService'].mean(), inplace = True)
df['FoodCourt'].fillna(df['FoodCourt'].mean(), inplace = True)
df['ShoppingMall'].fillna(df['ShoppingMall'].mean(), inplace = True)
df['Spa'].fillna(df['Spa'].mean(), inplace = True)
df['VRDeck'].fillna(df['VRDeck'].mean(), inplace = True)
df['Cabin_1'].fillna(df['Cabin_1'].mean(), inplace = True)            

C:\Users\arvin\AppData\Local\Temp\ipykernel_3536\2667939765.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train.append(test,ignore_index=True)


In [6]:
df['Cabin_2'].mode()

0    S
Name: Cabin_2, dtype: object

In [7]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Cabin_0', 'Cabin_1', 'Cabin_2', 'Passenger',
       'Passenger1', 'Age_0_1', 'Age_1_2', 'Age_2_3', 'Age_3_4', 'Age_4_5',
       'Age_5_6', 'Age_6_7', 'Age_7_8', 'Age_8_9', 'Age_9_10', 'Age_10_11',
       'Age_11_12', 'Age_12_13', 'Age_13_14', 'Age_14_15', 'Age_15_16',
       'Age_16_17', 'Age_17_18', 'Age_18_19', 'Age_19_20', 'Age_20_21',
       'Age_21_22', 'Age_22_23', 'Age_23_24', 'Age_24_25', 'Age_25_26',
       'Age_26_27', 'Age_27_28', 'Age_28_29', 'Age_29_30', 'Age_30_31',
       'Age_31_32', 'Age_32_33', 'Age_33_34', 'Age_34_35', 'Age_35_36',
       'Age_36_37', 'Age_37_38', 'Age_38_39', 'Age_39_40', 'Age_40_41',
       'Age_41_42', 'Age_42_43', 'Age_43_44', 'Age_44_45', 'Age_45_46',
       'Age_46_47', 'Age_47_48', 'Age_48_49', 'Age_49_50', 'Age_50_51',
       'Age_51_52', 'Age_52_53', 'Age_53_

In [8]:
import sklearn
df_categorical = df[[ 'HomePlanet', 'CryoSleep', 'Destination','Cabin_0','Cabin_2']]
df_numerical = df[[ 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Cabin_1']]
df_numerical_ = pd.DataFrame(sklearn.preprocessing.normalize(df_numerical),columns = [ 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Cabin_1'])
df_rest = df.drop([ 'HomePlanet', 'CryoSleep', 'Destination','Cabin_0',  'Cabin_2','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Cabin_1'],axis =1 )


In [9]:
df_categorical_out = pd.get_dummies(df_categorical)
df_ = pd.concat([df_rest, df_categorical_out], axis=1)
df = pd.concat([df_, df_numerical_], axis=1)

In [10]:
train = df[df['Transported']!='X'].reset_index(drop=True)
test_ = df[df['Transported']=='X'].reset_index(drop=True)

In [11]:
df['PassengerId']

0        0001_01
1        0002_01
2        0003_01
3        0003_02
4        0004_01
          ...   
12965    9266_02
12966    9269_01
12967    9271_01
12968    9273_01
12969    9277_01
Name: PassengerId, Length: 12970, dtype: object

In [12]:
drop_columns = ['PassengerId', 'Cabin', 'Age', 'Name', 'Transported']
label = train['Transported'].astype(int)
train = train.drop(drop_columns,axis = 1)

test_s = test_.drop(drop_columns,axis = 1)

In [13]:
label.value_counts()

1    4378
0    4315
Name: Transported, dtype: int64

In [14]:

import xgboost as xgb
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(train, label)
preds = xgb_cl.predict(test_s)

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\Loc

[03:32:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
test_s['Passenger'].value_counts()

1    3063
2     723
3     269
4     107
5      56
6      33
7      20
8       6
Name: Passenger, dtype: int64

In [16]:
test_['Transported']=preds==1

In [17]:
fill_subset = pd.DataFrame()
fill_subset['PassengerId']=list(set(test['PassengerId']) - set(test_['PassengerId'])) 
fill_subset['Transported']= False

test_[['PassengerId','Transported']].append(fill_subset,ignore_index=True).reset_index(drop=True).to_csv('Result_3.csv',index=False)

C:\Users\arvin\AppData\Local\Temp\ipykernel_3536\2031637108.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_[['PassengerId','Transported']].append(fill_subset,ignore_index=True).reset_index(drop=True).to_csv('Result_3.csv',index=False)


In [18]:
fill_subset

,PassengerId,Transported


In [19]:
test_[['PassengerId','Transported']].append(fill_subset,ignore_index=True).reset_index(drop=True)

C:\Users\arvin\AppData\Local\Temp\ipykernel_3536\3649455761.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_[['PassengerId','Transported']].append(fill_subset,ignore_index=True).reset_index(drop=True)


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [20]:
test['Transported'].value_counts()

X    4277
Name: Transported, dtype: int64

In [21]:
test_[['PassengerId','Transported']]

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [25]:
df.columns

Index(['PassengerId', 'Cabin', 'Age', 'VIP', 'Name', 'Transported',
       'Passenger', 'Passenger1', 'Age_0_1', 'Age_1_2',
       ...
       'Cabin_0_G', 'Cabin_0_T', 'Cabin_2_P', 'Cabin_2_S', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_1'],
      dtype='object', length=111)